## Environment setup

In [1]:
import glob, os, time
from datetime import datetime
from functools import partial
import importlib



import numpy as np

import tensorflow as tf

2024-01-20 13:11:45.783516: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 13:11:45.783592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 13:11:45.785504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 13:11:45.801069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Networks definitions and adaptations

In [2]:
from architecture_definitions import *

dir_postfix = get_classic_nn_dir_postfix() 
    
print_Classic_NN()  

Classic NN definitions:
dense_layer1_size: 128
dense_layer2_size: 64
dense_layer3_size: 32
dense_layer4_size: 1
------------------------


### Training data set preparation

In [3]:
import io_functions as io
importlib.reload(io)

batchSize = 4096
nEpochs = 1

#trainDataDir = "/scratch_ssd/akalinow/ProgrammingProjects/MachineLearning/OMTF/data/18_12_2020/"   
trainDataDir = "/home/kbunkow/cms_data/OMTF_data_2020/18_12_2020/"
trainFileNames = glob.glob(trainDataDir+'OMTFHits_pats0x0003_oldSample_files_*_chunk_0.tfrecord.gzip')

trainDataDir = "/scratch_cmsse/alibordi/data/training/"
trainFileNames = glob.glob(trainDataDir+'*OneOverPt*tfrecord.gzip')

dataset = io.get_Classic_NN_dataset(batchSize, nEpochs, trainFileNames, isTrain=True)

print("dataset", dataset)

Reading data from files:
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_April4_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_Feb15_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_Feb22_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_April14_chunk_0_filtered.tfrecord.gzip
/scratch_cmsse/alibordi/data/training/SingleMu_OneOverPt_April20_chunk_0_filtered.tfrecord.gzip


2024-01-20 13:11:49.484927: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: UNKNOWN ERROR (34)


dataset <_ParallelMapDataset element_spec=(TensorSpec(shape=(4096, 37), dtype=tf.float16, name=None), (TensorSpec(shape=(4096,), dtype=tf.float16, name=None),), TensorSpec(shape=(4096,), dtype=tf.float16, name=None))>


### Model definition

In [4]:
import model_functions as models
importlib.reload(models)

import io_functions as io
importlib.reload(io)

networkInputSize = 2 * np.sum(io.getFeaturesMask()) + 1
loss_fn = 'mape'
loss_fn = loss_MAPE_MAE

model = models.get_Classic_NN(networkInputSize=networkInputSize, loss_fn=loss_fn)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pt_layer_1 (Dense)          (None, 128)               4864      
                                                                 
 pt_layer_2 (Dense)          (None, 64)                8256      
                                                                 
 pt_layer_3 (Dense)          (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 15233 (59.50 KB)
Trainable params: 15233 (59.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### The training loop

In [5]:
from keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Compute dtype: float16
Variable dtype: float32


In [6]:
%%time
current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training start. Current Time =", current_time)

nEpochs = 150

log_dir = "logs/fit/" + current_time + dir_postfix
job_dir = "training/" + current_time + dir_postfix

checkpoint_path = job_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq = 5085)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=(10, 20))
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, verbose=1)
callbacks =  [tensorboard_callback, cp_callback, early_stop_callback]

model.save_weights(checkpoint_path.format(epoch=0))
   
model.fit(dataset.skip(10), 
          epochs=nEpochs, shuffle=True,
          callbacks=callbacks,
          validation_data = dataset.take(10)
            )
model.save(job_dir, save_format='tf')

current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Training end. Current Time =", current_time)

Training start. Current Time = 2024_Jan_20_13_11_50


2024-01-20 13:11:50.691252: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-01-20 13:11:50.691289: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-01-20 13:11:50.691446: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


Epoch 1/150
     10/Unknown - 3s 81ms/step - loss: 1608.8113

2024-01-20 13:11:53.868488: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-01-20 13:11:53.868540: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


     19/Unknown - 4s 104ms/step - loss: 986.3631 

2024-01-20 13:11:55.257398: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-01-20 13:11:57.207490: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-01-20 13:11:57.208676: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/fit/2024_Jan_20_13_11_50_classic_128_64_32_1/plugins/profile/2024_01_20_13_11_57/zcobl3.xplane.pb


1336/1336 [==============================] - 132s 97ms/step - loss: 133.5154 - val_loss: 90.1773
Epoch 2/150
1336/1336 [==============================] - 133s 98ms/step - loss: 86.9931 - val_loss: 81.1932
Epoch 3/150
1336/1336 [==============================] - 128s 95ms/step - loss: 80.3108 - val_loss: 77.0453
Epoch 4/150
1076/1336 [=======================>......] - ETA: 24s - loss: 76.2999
Epoch 4: saving model to training/2024_Jan_20_13_11_50_classic_128_64_32_1/cp-0004.ckpt
1336/1336 [==============================] - 129s 96ms/step - loss: 76.0023 - val_loss: 73.8493
Epoch 5/150
1336/1336 [==============================] - 131s 97ms/step - loss: 72.9864 - val_loss: 71.1113
Epoch 6/150
1336/1336 [==============================] - 132s 98ms/step - loss: 70.7908 - val_loss: 70.3947
Epoch 7/150
1336/1336 [==============================] - 131s 98ms/step - loss: 69.1169 - val_loss: 68.3575
Epoch 8/150
 817/1336 [=================>............] - ETA: 49s - loss: 67.9454
Epoch 8: saving

INFO:tensorflow:Assets written to: training/2024_Jan_20_13_11_50_classic_128_64_32_1/assets


Training end. Current Time = 2024_Jan_20_15_22_41
CPU times: user 9h 5min 36s, sys: 2h 6min 52s, total: 11h 12min 29s
Wall time: 2h 10min 51s
